# Exercícios de Fixação de Conceitos 1

### Índice
- Definição do problema de classificação</p>
- *Dataset*: consideraçõs e análise</p>
    - Estrutura do *dataset*
    - Representações *one-hot encoding* e categórica
    - Distribuição de frequências em função das classes (Histograma)
    - Visualização de imagens de dígitos manuscritos
- Conjuntos de treinamento e validação
- Classificador Linear
    - Modelo regularizado
    - Cálculo do erro quadrático médio
    - Cálculo do erro de classificação
    - Algoritmo para resolução do problema
- Resultados
    - Gráficos semilog do desempenho dos classificadores junto aos dados de validação para os 11 valores de coeficientes de regularização sugeridos
        - Desempenho baseado no erro quadrático médio
        - Desempenho baseado no erro de classificação
    - Gráficos semilog com uma busca refinida no entorno do mínimo dos gráficos anteriores
        - Desempenho baseado no erro quadrático médio
        - Desempenho baseado no erro de classificação
    - Valores dos 4 coeficientes de regularização encontrados na busca
    - Para o melhor classificador considerando o desempenho baseado no erro de classificação:
        - Obter a matriz de parâmetros do modelo
        - Gerar um gráfico de calor para cada um dos 10 classificadores lineares contidos no modelo
        - Matriz de confusão
        - Exemplos de dígitos classificados incorretamente

### Definição do problema

- O objetivo deste exercício consiste em sintetizar modelos lineares para classificação de padrões.
- É utilizado o famoso *dataset* de imagens de dígitos manuscritos [MNIST](http://yann.lecun.com/exdb/mnist/), o qual contém 60.000 amostras para treinamento e 10.000 amostras para teste.
- Cada imagem de entrada contém 784 pixels (no intervalo [0, 255], correspondente a níveis de cinza), visto que a dimensão é de 28x28 pixels.
- O problema de classificação investigado é do tipo multi-classe (10 classes), sendo que cada classe representa um dígito de 0 a 9.

### *Dataset*: considerações e análise
- Vamos compreender melhor a natureza dos dados.

In [15]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import random as rn
from sklearn.model_selection import train_test_split

In [16]:
a = range(0,101)

In [39]:
b = train_test_split(a, test_size=0.25, random_state=42)

True
26
[[40, 72, 9, 96, 11, 91, 64, 28, 83, 5, 47, 53, 35, 16, 81, 34, 7, 43, 73, 27, 19, 94, 25, 62, 49, 13, 24, 3, 17, 38, 8, 79, 6, 65, 36, 88, 56, 100, 54, 50, 68, 46, 69, 61, 98, 80, 41, 58, 48, 90, 57, 75, 32, 95, 59, 63, 85, 37, 29, 1, 52, 21, 2, 23, 87, 99, 74, 86, 82, 20, 60, 71, 14, 92, 51], [84, 55, 66, 67, 45, 39, 22, 44, 10, 0, 18, 30, 97, 33, 77, 4, 93, 78, 12, 31, 76, 89, 26, 42, 70, 15]]
